#### imports all in the first cell

In [9]:
import json
from pyspark.sql import Row


### Linux Commands

#### Bring up cluster and make sure it's running
```
docker-compose up -d
docker-compose ps
docker ps -a
```


#### Create the topic
```
docker-compose exec kafka kafka-topics --create --topic assessments --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181
```

#### Check the topic
```
docker-compose exec kafka kafka-topics --describe --topic assessments --zookeeper zookeeper:32181
```


#### Start the kafka logs
```
docker-compose logs -f kafka
```


#### Publish the json objects on the kafka topic
```
docker-compose exec mids bash -c "cat /w205/project-2-krc-generic-student/assessment-attempts-20180128-121051-nested.json | jq '.[]' -c | kafkacat -P -b kafka:29092 -t assessments"
```

#### Read the topic with kafkacat to make sure everything is on it
```
docker-compose exec mids bash -c "kafkacat -C -b kafka:29092 -t assessments -o beginning -e"
```

####  Shutdown the cluster and verify it's down
```
docker-compose down
docker-compose ps
docker ps -a
```


### week 7

In [2]:
messages = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load() 


In [3]:
messages.printSchema()


root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [4]:
messages.show()


+----+--------------------+-----------+---------+------+--------------------+-------------+
| key|               value|      topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----------+---------+------+--------------------+-------------+
|null|[7B 22 6B 65 65 6...|assessments|        0|     0|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     1|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     2|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     3|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     4|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     5|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     6|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     7|1969-12-31 23:59:...|   

In [5]:
messages_as_strings=messages.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")


In [6]:
messages_as_strings.show()


+----+--------------------+
| key|               value|
+----+--------------------+
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
+----+--------------------+
only showing top 20 rows



In [7]:
messages_as_strings.printSchema()


root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



In [8]:
messages_as_strings.count()


3280

In [9]:
messages_as_strings.select('value').take(1)


[Row(value='{"keen_timestamp":"1516717442.735266","max_attempts":"1.0","started_at":"2018-01-23T14:23:19.082Z","base_exam_id":"37f0a30a-7464-11e6-aa92-a8667f27e5dc","user_exam_id":"6d4089e4-bde5-4a22-b65f-18bce9ab79c8","sequences":{"questions":[{"user_incomplete":true,"user_correct":false,"options":[{"checked":true,"at":"2018-01-23T14:23:24.670Z","id":"49c574b4-5c82-4ffd-9bd1-c3358faf850d","submitted":1,"correct":true},{"checked":true,"at":"2018-01-23T14:23:25.914Z","id":"f2528210-35c3-4320-acf3-9056567ea19f","submitted":1,"correct":true},{"checked":false,"correct":true,"id":"d1bf026f-554f-4543-bdd2-54dcf105b826"}],"user_submitted":true,"id":"7a2ed6d3-f492-49b3-b8aa-d080a8aad986","user_result":"missed_some"},{"user_incomplete":false,"user_correct":false,"options":[{"checked":true,"at":"2018-01-23T14:23:30.116Z","id":"a35d0e80-8c49-415d-b8cb-c21a02627e2b","submitted":1},{"checked":false,"correct":true,"id":"bccd6e2e-2cef-4c72-8bfa-317db0ac48bb"},{"checked":true,"at":"2018-01-23T14:23:41

In [10]:
messages_as_strings.select('value').take(1)[0].value


'{"keen_timestamp":"1516717442.735266","max_attempts":"1.0","started_at":"2018-01-23T14:23:19.082Z","base_exam_id":"37f0a30a-7464-11e6-aa92-a8667f27e5dc","user_exam_id":"6d4089e4-bde5-4a22-b65f-18bce9ab79c8","sequences":{"questions":[{"user_incomplete":true,"user_correct":false,"options":[{"checked":true,"at":"2018-01-23T14:23:24.670Z","id":"49c574b4-5c82-4ffd-9bd1-c3358faf850d","submitted":1,"correct":true},{"checked":true,"at":"2018-01-23T14:23:25.914Z","id":"f2528210-35c3-4320-acf3-9056567ea19f","submitted":1,"correct":true},{"checked":false,"correct":true,"id":"d1bf026f-554f-4543-bdd2-54dcf105b826"}],"user_submitted":true,"id":"7a2ed6d3-f492-49b3-b8aa-d080a8aad986","user_result":"missed_some"},{"user_incomplete":false,"user_correct":false,"options":[{"checked":true,"at":"2018-01-23T14:23:30.116Z","id":"a35d0e80-8c49-415d-b8cb-c21a02627e2b","submitted":1},{"checked":false,"correct":true,"id":"bccd6e2e-2cef-4c72-8bfa-317db0ac48bb"},{"checked":true,"at":"2018-01-23T14:23:41.791Z","id"

In [11]:
first_message=json.loads(messages_as_strings.select('value').take(1)[0].value)


In [12]:
type(first_message)

dict

In [13]:
first_message

{'base_exam_id': '37f0a30a-7464-11e6-aa92-a8667f27e5dc',
 'certification': 'false',
 'exam_name': 'Normal Forms and All That Jazz Master Class',
 'keen_created_at': '1516717442.735266',
 'keen_id': '5a6745820eb8ab00016be1f1',
 'keen_timestamp': '1516717442.735266',
 'max_attempts': '1.0',
 'sequences': {'attempt': 1,
  'counts': {'all_correct': False,
   'correct': 2,
   'incomplete': 1,
   'incorrect': 1,
   'submitted': 4,
   'total': 4,
   'unanswered': 0},
  'id': '5b28a462-7a3b-42e0-b508-09f3906d1703',
  'questions': [{'id': '7a2ed6d3-f492-49b3-b8aa-d080a8aad986',
    'options': [{'at': '2018-01-23T14:23:24.670Z',
      'checked': True,
      'correct': True,
      'id': '49c574b4-5c82-4ffd-9bd1-c3358faf850d',
      'submitted': 1},
     {'at': '2018-01-23T14:23:25.914Z',
      'checked': True,
      'correct': True,
      'id': 'f2528210-35c3-4320-acf3-9056567ea19f',
      'submitted': 1},
     {'checked': False,
      'correct': True,
      'id': 'd1bf026f-554f-4543-bdd2-54dcf10

In [14]:
print(first_message['sequences']['counts']['correct'])


2


### week 8 (from my starter code)

In [1]:
raw_assessments = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load() 


In [2]:
raw_assessments.cache()


DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [3]:
raw_assessments.show()

+----+--------------------+-----------+---------+------+--------------------+-------------+
| key|               value|      topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----------+---------+------+--------------------+-------------+
|null|[7B 22 6B 65 65 6...|assessments|        0|     0|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     1|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     2|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     3|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     4|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     5|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     6|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     7|1969-12-31 23:59:...|   

In [4]:
assessments = raw_assessments.select(raw_assessments.value.cast('string'))


In [5]:
assessments.show()

+--------------------+
|               value|
+--------------------+
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
+--------------------+
only showing top 20 rows



In [10]:
extracted_assessments = assessments.rdd.map(lambda x: Row(**json.loads(x.value))).toDF()


In [11]:
extracted_assessments.show()

+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+
|        base_exam_id|certification|           exam_name|   keen_created_at|             keen_id|    keen_timestamp|max_attempts|           sequences|          started_at|        user_exam_id|
+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717442.735266|5a6745820eb8ab000...| 1516717442.735266|         1.0|Map(questions -> ...|2018-01-23T14:23:...|6d4089e4-bde5-4a2...|
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717377.639827|5a674541ab6b0a000...| 1516717377.639827|         1.0|Map(questions -> ...|2018-01-23T14:21:...|2fec1534-b41f-441...|
|4beeac16-bb83-4d5...|        false

In [12]:
extracted_assessments.registerTempTable('assessments')


In [13]:
spark.sql("select keen_id from assessments limit 10").show()


+--------------------+
|             keen_id|
+--------------------+
|5a6745820eb8ab000...|
|5a674541ab6b0a000...|
|5a67999d3ed3e3000...|
|5a6799694fc7c7000...|
|5a6791e824fccd000...|
|5a67a0b6852c2a000...|
|5a67b627cc80e6000...|
|5a67ac8cb0a5f4000...|
|5a67a9ba060087000...|
|5a67ac54411aed000...|
+--------------------+



In [14]:
spark.sql("select keen_timestamp, sequences.questions[0].user_incomplete from assessments limit 10").show()


+------------------+-------------------------------------------------------+
|    keen_timestamp|sequences[questions] AS `questions`[0][user_incomplete]|
+------------------+-------------------------------------------------------+
| 1516717442.735266|                                                   true|
| 1516717377.639827|                                                  false|
| 1516738973.653394|                                                  false|
|1516738921.1137421|                                                  false|
| 1516737000.212122|                                                  false|
| 1516740790.309757|                                                  false|
|1516746279.3801291|                                                  false|
| 1516743820.305464|                                                  false|
|  1516743098.56811|                                                  false|
| 1516743764.813107|                                                  false|

### Nested multi-valued as a dictionary (from starter code)

In [26]:
def my_lambda_sequences_id(x):
    raw_dict = json.loads(x.value)
    my_dict = {"keen_id" : raw_dict["keen_id"], "sequences_id" : raw_dict["sequences"]["id"]}
    return Row(**my_dict)


In [27]:
my_sequences = assessments.rdd.map(my_lambda_sequences_id).toDF()


In [28]:
my_sequences.registerTempTable('sequences')


In [29]:
spark.sql("select sequences_id from sequences limit 10").show()


+--------------------+
|        sequences_id|
+--------------------+
|5b28a462-7a3b-42e...|
|5b28a462-7a3b-42e...|
|b370a3aa-bf9e-4c1...|
|b370a3aa-bf9e-4c1...|
|04a192c1-4f5c-4ac...|
|e7110aed-0d08-4cb...|
|5251db24-2a6e-424...|
|066b5326-e547-4da...|
|8ac691f8-8c1a-403...|
|066b5326-e547-4da...|
+--------------------+



In [30]:
spark.sql("select a.keen_id, a.keen_timestamp, s.sequences_id from assessments a join sequences s on a.keen_id = s.keen_id limit 10").show()


+--------------------+------------------+--------------------+
|             keen_id|    keen_timestamp|        sequences_id|
+--------------------+------------------+--------------------+
|5a17a67efa1257000...|1511499390.3836269|8ac691f8-8c1a-403...|
|5a26ee9cbf5ce1000...|1512500892.4166169|9bd87823-4508-4e0...|
|5a29dcac74b662000...|1512692908.8423469|e7110aed-0d08-4cb...|
|5a2fdab0eabeda000...|1513085616.2275269|cd800e92-afc3-447...|
|5a30105020e9d4000...|1513099344.8624721|8ac691f8-8c1a-403...|
|5a3a6fc3f0a100000...| 1513779139.354213|e7110aed-0d08-4cb...|
|5a4e17fe08a892000...|1515067390.1336551|9abd5b51-6bd8-11e...|
|5a4f3c69cc6444000...| 1515142249.858722|083844c5-772f-48d...|
|5a51b21bd0480b000...| 1515303451.773272|e7110aed-0d08-4cb...|
|5a575a85329e1a000...| 1515674245.348099|25ca21fe-4dbb-446...|
+--------------------+------------------+--------------------+



### Nested multi-valued as a list (from starter code)

In [31]:
def my_lambda_questions(x):
    raw_dict = json.loads(x.value)
    my_list = []
    my_count = 0
    for l in raw_dict["sequences"]["questions"]:
        my_count += 1
        my_dict = {"keen_id" : raw_dict["keen_id"], "my_count" : my_count, "id" : l["id"]}
        my_list.append(Row(**my_dict))
    return my_list


In [32]:
my_questions = assessments.rdd.flatMap(my_lambda_questions).toDF()


In [33]:
my_questions.registerTempTable('questions')


In [34]:
spark.sql("select id, my_count from questions limit 10").show()


+--------------------+--------+
|                  id|my_count|
+--------------------+--------+
|7a2ed6d3-f492-49b...|       1|
|bbed4358-999d-446...|       2|
|e6ad8644-96b1-461...|       3|
|95194331-ac43-454...|       4|
|95194331-ac43-454...|       1|
|bbed4358-999d-446...|       2|
|e6ad8644-96b1-461...|       3|
|7a2ed6d3-f492-49b...|       4|
|b9ff2e88-cf9d-4bd...|       1|
|bec23e7b-4870-49f...|       2|
+--------------------+--------+



In [35]:
spark.sql("select q.keen_id, a.keen_timestamp, q.id from assessments a join questions q on a.keen_id = q.keen_id limit 10").show()


+--------------------+------------------+--------------------+
|             keen_id|    keen_timestamp|                  id|
+--------------------+------------------+--------------------+
|5a17a67efa1257000...|1511499390.3836269|803fc93f-7eb2-412...|
|5a17a67efa1257000...|1511499390.3836269|f3cb88cc-5b79-41b...|
|5a17a67efa1257000...|1511499390.3836269|32fe7d8d-6d89-4db...|
|5a17a67efa1257000...|1511499390.3836269|5c34cf19-8cfd-4f5...|
|5a26ee9cbf5ce1000...|1512500892.4166169|0603e6f4-c3f9-4c2...|
|5a26ee9cbf5ce1000...|1512500892.4166169|26a06b88-2758-45b...|
|5a26ee9cbf5ce1000...|1512500892.4166169|25b6effe-79b0-4c4...|
|5a26ee9cbf5ce1000...|1512500892.4166169|6de03a9b-2a78-46b...|
|5a26ee9cbf5ce1000...|1512500892.4166169|aaf39991-fa83-470...|
|5a26ee9cbf5ce1000...|1512500892.4166169|aab2e817-73dc-4ff...|
+--------------------+------------------+--------------------+



### Handling holes in json data (from starter code)

In [36]:
def my_lambda_correct_total(x):
    
    raw_dict = json.loads(x.value)
    my_list = []
    
    if "sequences" in raw_dict:   
        
        if "counts" in raw_dict["sequences"]:
            
            if "correct" in raw_dict["sequences"]["counts"] and "total" in raw_dict["sequences"]["counts"]:
                    
                my_dict = {"correct": raw_dict["sequences"]["counts"]["correct"], 
                           "total": raw_dict["sequences"]["counts"]["total"]}
                my_list.append(Row(**my_dict))
    
    return my_list

In [54]:
def my_lambda_correct_total(x):
    
    raw_dict = json.loads(x.value)
    my_list = []
    
    if "sequences" in raw_dict:   
        
        if "counts" in raw_dict["sequences"]:
            
            if "correct" in raw_dict["sequences"]["counts"] and "total" in raw_dict["sequences"]["counts"]:
                    
                my_dict = {"exam_name": raw_dict["exam_name"],
                           "correct": raw_dict["sequences"]["counts"]["correct"], 
                           "total": raw_dict["sequences"]["counts"]["total"]}
                my_list.append(Row(**my_dict))
    
    return my_list

In [55]:
my_correct_total = assessments.rdd.flatMap(my_lambda_correct_total).toDF()


In [56]:
my_correct_total.registerTempTable('ct')


In [59]:
spark.sql("select exam_name, correct, total from ct limit 10").show(truncate=False)


+-------------------------------------------+-------+-----+
|exam_name                                  |correct|total|
+-------------------------------------------+-------+-----+
|Normal Forms and All That Jazz Master Class|2      |4    |
|Normal Forms and All That Jazz Master Class|1      |4    |
|The Principles of Microservices            |3      |4    |
|The Principles of Microservices            |2      |4    |
|Introduction to Big Data                   |3      |4    |
|Learning Git                               |5      |5    |
|Git Fundamentals for Web Developers        |1      |1    |
|Introduction to Python                     |5      |5    |
|Intermediate Python Programming            |4      |4    |
|Introduction to Python                     |0      |5    |
+-------------------------------------------+-------+-----+



In [40]:
spark.sql("select correct / total as score from ct limit 10").show()


+-----+
|score|
+-----+
|  0.5|
| 0.25|
| 0.75|
|  0.5|
| 0.75|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  0.0|
+-----+



In [41]:
spark.sql("select avg(correct / total)*100 as avg_score from ct limit 10").show()


+-----------------+
|        avg_score|
+-----------------+
|62.65699745547047|
+-----------------+



In [42]:
spark.sql("select stddev(correct / total) as standard_deviation from ct limit 10").show()


+-------------------+
| standard_deviation|
+-------------------+
|0.31086692286170553|
+-------------------+



In [60]:
def my_lambda_exam_takers(x):
    
    raw_dict = json.loads(x.value)
    my_list = []
    
    if "sequences" in raw_dict:   
        
        if "counts" in raw_dict["sequences"]:
                    
            my_dict = {"exam_name": raw_dict["exam_name"],
                       "taken": "true"}
        else:
            my_dict = {"exam_name": raw_dict["exam_name"],
                       "taken": "false"}
            
        my_list.append(Row(**my_dict))
    
    return my_list

In [61]:
my_exam_takers = assessments.rdd.flatMap(my_lambda_exam_takers).toDF()

In [62]:
my_exam_takers.registerTempTable('exam_takers')


In [68]:
spark.sql("select exam_name, taken, count(*) from exam_takers where taken = false group by exam_name, taken ").show(truncate=False)

+---------------------------------------------------+-----+--------+
|exam_name                                          |taken|count(1)|
+---------------------------------------------------+-----+--------+
|Example Exam For Development and Testing oh yeahsdf|false|5       |
+---------------------------------------------------+-----+--------+



### Write dataframes out to Hadoop HDFS in Parquet format to create a batch and serving layers scale up SQL

In [43]:
raw_assessments.write.mode('overwrite').parquet("/tmp/raw_assessments")

assessments.write.mode('overwrite').parquet("/tmp/assessments")

extracted_assessments.write.mode('overwrite').parquet("/tmp/extracted_assessments")

my_sequences.write.mode('overwrite').parquet("/tmp/sequences")

my_questions.write.mode('overwrite').parquet("/tmp/questions")

my_correct_total.write.mode('overwrite').parquet("/tmp/correct_total")



### Business Questions and Answers Using Spark SQL against MPP Dataframe/RDD in Memory


### How many assessments are in the dataset?

In [44]:
spark.sql("select count(*) as number_of_assessments from assessments").show()

+---------------------+
|number_of_assessments|
+---------------------+
|                 3280|
+---------------------+



### What's the name of your Kafka topic? How did you come up with that name?

assessments

Subject matter was assessments

### How many people took Learning Git?

In [45]:
spark.sql("select count(*) as students_who_took_learning_git  from assessments where exam_name like 'Learning Git' ").show()

+------------------------------+
|students_who_took_learning_git|
+------------------------------+
|                           394|
+------------------------------+



### What is the least common course taken? And the most common?

In [46]:
spark.sql("select exam_name, count(*) as number_students  from assessments group by exam_name order by 2 ").show()

+--------------------+---------------+
|           exam_name|number_students|
+--------------------+---------------+
|Learning to Visua...|              1|
|Native Web Apps f...|              1|
|Nulls, Three-valu...|              1|
|Operating Red Hat...|              1|
|The Closed World ...|              2|
|What's New in Jav...|              2|
|Understanding the...|              2|
|Client-Side Data ...|              2|
|Learning Spring P...|              2|
|Hibernate and JPA...|              2|
|Arduino Prototypi...|              2|
| Mastering Web Views|              3|
|Building Web Serv...|              3|
|Getting Ready for...|              3|
|Using Web Components|              3|
|Service Based Arc...|              3|
|       View Updating|              4|
|Using Storytellin...|              4|
|Example Exam For ...|              5|
|Starting a Grails...|              5|
+--------------------+---------------+
only showing top 20 rows



In [47]:
spark.sql("select exam_name, count(*) as number_students  from assessments group by exam_name order by 2 desc ").show()

+--------------------+---------------+
|           exam_name|number_students|
+--------------------+---------------+
|        Learning Git|            394|
|Introduction to P...|            162|
|Introduction to J...|            158|
|Intermediate Pyth...|            158|
|Learning to Progr...|            128|
|Introduction to M...|            119|
|Software Architec...|            109|
|Beginning C# Prog...|             95|
|    Learning Eclipse|             85|
|Learning Apache M...|             80|
|Beginning Program...|             79|
|       Mastering Git|             77|
|Introduction to B...|             75|
|Advanced Machine ...|             67|
|Learning Linux Sy...|             59|
|JavaScript: The G...|             58|
|        Learning SQL|             57|
|Practical Java Pr...|             53|
|    HTML5 The Basics|             52|
|   Python Epiphanies|             51|
+--------------------+---------------+
only showing top 20 rows



### Write a query, save that query to a new dataframe, register that dataframe as a temp table, then execute a query against the new temp table

In [48]:
my_scores = spark.sql("select correct / total as score from ct")

In [49]:
my_scores.show()

+------------------+
|             score|
+------------------+
|               0.5|
|              0.25|
|              0.75|
|               0.5|
|              0.75|
|               1.0|
|               1.0|
|               1.0|
|               1.0|
|               0.0|
|              0.75|
|               1.0|
|0.6666666666666666|
|0.6666666666666666|
|               0.8|
|              0.75|
|              0.75|
|               1.0|
|               0.5|
|               1.0|
+------------------+
only showing top 20 rows



In [50]:
my_scores.registerTempTable("scores")

In [51]:
my_scores_gt_70_pct = spark.sql("select * from scores where score > 0.7 ")

In [52]:
my_scores_gt_70_pct.registerTempTable("scores_gt_70")

In [53]:
spark.sql("select * from scores_gt_70 ").show()

+-----+
|score|
+-----+
| 0.75|
| 0.75|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
| 0.75|
|  1.0|
|  0.8|
| 0.75|
| 0.75|
|  1.0|
|  1.0|
| 0.75|
| 0.75|
| 0.75|
|  1.0|
|  1.0|
|  1.0|
| 0.75|
+-----+
only showing top 20 rows

